## import Libraries

In [1]:
import nltk 
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords 
import numpy as np
import tensorflow 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import json
import pickle
import random
from time import sleep

## Data

In [2]:
with open("../Data/Raw/Intentss.json", encoding="utf-8") as file:  #load the json file content
    data = json.load(file)


## NLP

In [3]:
words=[]
classes = []
documents = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
             classes.append(intent['tag'])

In [4]:
stop_words= list(stopwords.words("Arabic"))  #save the arabic stop words in a list


In [5]:
[words.remove(w) for w in words if w in stop_words] 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
stemmer = ISRIStemmer() # Stemmer function 
ignore_words = ['?', '!']
words = [stemmer.stem(w) for w in words if w not in ignore_words] #apply Stemmer on the words included in the list
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('../Data/Processed/words1.pkl','wb'))
pickle.dump(classes,open('../Data/Processed/classes1.pkl','wb'))
pickle.dump(data,open('../Data/Processed/intents1.pkl','wb'))

408 documents
109 classes ['  التهاب الشبكية', '  انسداد الرئوي المزمن', '  وجع البطن', ' الاكزيما', ' الثآليل', ' الغريق', ' سرطان الجلد', ' كرون', 'آلام الظهر', 'آلام المفاصل', 'أزمة القلب', 'إكزيما', 'إنعاش القلب والرئة', 'اختناق', 'ارتجاع الصمامات القلبية', 'ارتفاع الضغط', 'ارتفاع ضغط الدم', 'اضطرابات هضمية', 'الأكل و المشروبات الممنوعة لمريض السكر', 'الأكل و المشروبات الممنوعة لمريض الضغط', 'الأنيميا', 'الإسعافات الأولية لجروح عضات أو خدوش الكلاب', 'الإسعافات الأولية لضغط الدم المنخفض', 'الإسعافات الأولية للإختناق', 'الإسعافات الأولية للإغماء', 'الإسعافات الأولية للحروق', 'الإسعافات الأولية للسكتة القلبية', 'الإسعافات الأولية للصدمة', 'الإسعافات الأولية للنزيف', 'الإسعافات الأولية لمرض اضطراب نقص الانتباه وفرط الحركة (ADHD)', 'الاسعافات الاولية للكسور', 'الاسعافات الاوليه', 'الاسهال', 'الامساك', 'الانسداد الشعب الهوائية المزمن', 'التصبغ البقعي', 'التصلب الجانبي الضموري ', 'التصلب العصبي المتعدد', 'التعامل مع مريض السرطان', 'التهاب الأمعاء', 'التهاب الجيوب الأنفية', 'التهاب الحنجرة

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(w) for w in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        if w in pattern_words:
            bag.append(1)
            
        else:
            bag.append(0)     
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    #print('classes.index(doc[1])',classes.index(doc[1]))
    #print('output_row',output_row)
    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
training

Training data created


C:\Users\ibrah\AppData\Local\Temp\ipykernel_8944\3510048649.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
training.shape

(408, 2)

## NN model

In [9]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(164, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(120, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model

hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('../Models/model.h5', hist)
print("model created")

Epoch 1/100
82/82 [==============================] - 1s 3ms/step - loss: 4.6546 - accuracy: 0.0245
Epoch 2/100
82/82 [==============================] - 0s 2ms/step - loss: 4.4031 - accuracy: 0.1054
Epoch 3/100
82/82 [==============================] - 0s 2ms/step - loss: 4.0393 - accuracy: 0.1618
Epoch 4/100
82/82 [==============================] - 0s 3ms/step - loss: 3.7166 - accuracy: 0.2108
Epoch 5/100
82/82 [==============================] - 0s 2ms/step - loss: 3.3120 - accuracy: 0.2941
Epoch 6/100
82/82 [==============================] - 0s 2ms/step - loss: 3.0009 - accuracy: 0.3309
Epoch 7/100
82/82 [==============================] - 0s 2ms/step - loss: 2.6515 - accuracy: 0.4069
Epoch 8/100
82/82 [==============================] - 0s 3ms/step - loss: 2.3226 - accuracy: 0.4657
Epoch 9/100
82/82 [==============================] - 0s 2ms/step - loss: 2.0706 - accuracy: 0.4877
Epoch 10/100
82/82 [==============================] - 0s 2ms/step - loss: 1.9029 - accuracy: 0.5343
Epoch 11/

82/82 [==============================] - 0s 2ms/step - loss: 0.2385 - accuracy: 0.8873
Epoch 84/100
82/82 [==============================] - 0s 2ms/step - loss: 0.2175 - accuracy: 0.8946
Epoch 85/100
82/82 [==============================] - 0s 2ms/step - loss: 0.2027 - accuracy: 0.9142
Epoch 86/100
82/82 [==============================] - 0s 2ms/step - loss: 0.2138 - accuracy: 0.8922
Epoch 87/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1969 - accuracy: 0.9044
Epoch 88/100
82/82 [==============================] - 0s 2ms/step - loss: 0.2100 - accuracy: 0.8946
Epoch 89/100
82/82 [==============================] - 0s 2ms/step - loss: 0.2178 - accuracy: 0.8897
Epoch 90/100
82/82 [==============================] - 0s 2ms/step - loss: 0.2368 - accuracy: 0.8922
Epoch 91/100
82/82 [==============================] - 0s 2ms/step - loss: 0.1886 - accuracy: 0.9044
Epoch 92/100
82/82 [==============================] - 0s 2ms/step - loss: 0.2077 - accuracy: 0.8922
Epoch 93/100


##  CHATBOT

In [10]:
from keras.models import load_model

# Load the saved model
model = load_model('../Models/model.h5')

In [11]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(w) for w in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, data):
    tag = ints[0]['intent']
    list_of_intents = data['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result,tag


def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, data)
    return res

In [12]:
def chat(): 
    print("انا دكتور الطوارئ كيف يمكننى مساعدتك")
    while True:
        print("انت:")
        inp = input() 
        results,tag = chatbot_response(inp) 
        sleep(1.5)
        print(results)
        if  tag=="توديع":
            break

In [13]:
chat()

انا دكتور الطوارئ كيف يمكننى مساعدتك
انت:
سلام
1/1 [==============================] - 0s 79ms/step
مع السلامة 
